In [1]:
from mixes import *

# Concepts

Mixes ...

## Components

Components ...

The base `Component` class is meant to usable generically.  For example, we might have a solution of MgCl₂ that we'd like to use to make a Mg-added buffer.

In [2]:
mg = Component("MgCl₂", "1 M")
print(mg)

Component(name='MgCl₂', concentration=<Quantity(1, 'molar')>, plate='', well=None)


Like many components in mixes, the concentration is easiest to enter as a string.  mixes uses the [pint](https://github.com/hgrecco/pint) library to handle units, and Python's [decimal](FIXME) library to avoid imprecision in calculations.  It does this as transparently as possible: you can enter most values with units as either a string, or a pint Quantity, and they will be converted correctly.  You can use `Q_` as a shorthand to create a Quantity from a string, or a number for the value and string for the units.  The input should be quite flexible, for example:

In [3]:
Q_("5 µM") == Q_(5, "µM") == Q_("5 micromolar") == Q_("5 pmol / microliter")

True

In addition to having a name and a concentration, a component can have a location (currently using the `plate` property), and, if the location is a plate name, can also be given a `well`.

## Actions and Mixes

mixes combines Components into Mixes through Actions.  Actions specify what we'd like to do with a component, or a list of components, when we add them to a mix.  For example, we might want to make a buffer stock with 125 mM of MgCl₂ in it, in which case we could use the `FixedConcentration` action, which adds a single component at a fixed target concentration:

In [6]:
add_mg = FixedConcentration(mg, "125 mM")

FixedConcentration(component=Component(name='MgCl₂', concentration=<Quantity(1, 'molar')>, plate='', well=None), fixed_concentration=<Quantity(125, 'millimolar')>)


A `Mix`, then, is a list of these actions, together with some overall properties, like a name or 

In [9]:
mg_buffer = Mix([add_mg], "10× Mg", fixed_total_volume="1 mL")

In [10]:
mg_buffer

Table: Mix: 10× Mg, Conc: 125.00 mM, Total Vol: 1.00 ml

| Component   | [Src]   | [Dest]      | #   | Ea Tx Vol   | Tot Tx Vol   | Location   | Note   |
|:------------|:--------|:------------|:----|:------------|:-------------|:-----------|:-------|
| MgCl₂       | 1.00 M  | 125.00 mM   |     | 125.00 µl   | 125.00 µl    |            |        |
| Buffer      |         |             |     | 875.00 µl   | 875.00 µl    |            |        |
| *Total:*    |         | *125.00 mM* | *2* |             | *1.00 ml*    |            |        |

As we will see later, a `Mix` itself can also be a component in other mixes.

# Strands and References

A `Strand` is a type of component that also keeps track of a sequence:

In [14]:
Strand("S1", concentration="100 µM", sequence="AGAAT")

Strand(name='S1', concentration=<Quantity(100, 'micromolar')>, plate='', well=None, sequence='AGAAT')

Specifying all properties of every component in code would be time consuming and error prone.  Instead, we can specify the components without all properties, or even with just a name, and then use a `Reference` to add information to them.  Here, we'll create a simple reference as (fake) csv file:

In [20]:
import io
# Columns are "Name", "Plate", "Well", "Concentration (nM)", "Sequence"
csv_file = io.StringIO("""
Name,Plate,Well,"Concentration (nM)",Sequence
S1,plate1,A2,100000,AGAAT
S2,plate1,A3,125000,GTTCT
""")

ref = Reference.from_csv(csv_file)

Now, we can use `.with_reference` to add information:

In [21]:
Strand("S2")

Strand(name='S2', concentration=<Quantity(NaN, 'nanomolar')>, plate='', well=None, sequence=None)

In [22]:
Strand("S2").with_reference(ref)

Strand(name='S2', concentration=<Quantity(125.000000, 'micromolar')>, plate='plate1', well=WellPos("A3"), sequence='GTTCT')

## Larger mixes

In [35]:
ref = ref

In [33]:
strandmix1 = Mix(
    [MultiFixedVolume(
        components=[Strand(f"S{x}") for x in range(0,10)],
        fixed_volume="2 µL"
    )], "strand mix A"
).with_reference(ref)
strandmix1

In [34]:
strandmix2 = Mix(
    [MultiFixedConcentration(
        components=[Strand(f"S{x}") for x in range(10,20)],
        fixed_concentration="1 µM"
    )], "strand mix B"
).with_reference(ref)
strandmix2

With strands on plates...

In [39]:
sample1 = Mix(
    [
        FixedConcentration(strandmix1, "500 nM"),
        FixedConcentration(strandmix2, "100 nM"),
        FixedConcentration(mg_buffer, "12.5 mM")
    ],
    name = "Sample 1",
    fixed_total_volume = "100 µL"
)

In addition to seeing the series of recipes above...

In [38]:
sample1.all_components()

Init signature:
Mix(
    actions: 'Sequence[AbstractAction]',
    name: 'str',
    buffer_name: 'str' = 'Buffer',
    reference: 'Reference | None' = None,
    *,
    test_tube_name: 'str | None' = None,
    fixed_total_volume: 'str | pint.Quantity' = <Quantity(NaN, 'microliter')>,
    fixed_concentration: 'str | Quantity[Decimal] | None' = None,
    min_volume: 'str | pint.Quantity' = <Quantity(0.5, 'microliter')>,
) -> None
Docstring:     
Class denoting a Mix, a collection of source components mixed to
some volume or concentration.
Init docstring: Method generated by attrs for class Mix.
File:           ~/.local/lib/python3.10/site-packages/hieromix/mixes.py
Type:           ABCMeta
Subclasses:     
